In [3]:
pip install selenium webdriver-manager beautifulsoup4 pandas

Note: you may need to restart the kernel to use updated packages.


In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Configuración Selenium
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = "https://nuevavida-adopciones.org/perros-en-adopcion-protectora-madrid-nuevavida"
driver.get(url)
wait = WebDriverWait(driver, 10)

# Aceptar cookies
try:
    ok_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.wordpress-gdpr-popup-agree")))
    ok_button.click()
    print("✅ Cookies aceptadas.")
except:
    print("⚠️ No se encontró el popup de cookies.")

# Scroll para cargar los perros
def scroll_to_bottom(driver, pause_time=2, max_scrolls=10):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(max_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    print("✅ Scroll completado.")

# Extraer datos del perfil
def extraer_datos_perfil(driver):
    nombre = driver.find_element(By.CSS_SELECTOR, "h1.elementor-heading-title").text.strip()
    bloques_info = driver.find_elements(By.CSS_SELECTOR, "div.elementor-widget-container")

    info = {
        "sexo": "",
        "edad": "",
        "tamaño": "",
        "raza": "",
        "descripcion": "",
        "compatible_con_niños": "ND"
    }

    descripcion_encontrada = False

    for el in bloques_info:
        text = el.text.strip()
        if text.startswith("Sexo:"):
            info["sexo"] = text.replace("Sexo:", "").strip()
        elif text.startswith("Edad:"):
            info["edad"] = text.replace("Edad:", "").strip()
        elif text.startswith("Tamaño:"):
            info["tamaño"] = text.replace("Tamaño:", "").strip()
        elif "cruce" in text.lower() or "raza" in text.lower():
            info["raza"] = text.strip()
        elif f"Sobre {nombre}" in text:
            descripcion_encontrada = True
        elif descripcion_encontrada and len(text) > 30:
            info["descripcion"] += text + "\n"

    texto_completo = info["descripcion"].lower()
    if "niños" in texto_completo or "niñas" in texto_completo:
        info["compatible_con_niños"] = "Sí"
    elif "no compatible con niños" in texto_completo:
        info["compatible_con_niños"] = "No"

    return {
        "nombre": nombre,
        "sexo": info["sexo"],
        "edad": info["edad"],
        "tamaño": info["tamaño"],
        "raza": info["raza"],
        "descripcion": info["descripcion"].strip(),
        "compatible_con_niños": info["compatible_con_niños"],
        "url": driver.current_url
    }

# Scrapea todos los perros en la página actual
def scrape_pagina_actual():
    scroll_to_bottom(driver)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.jet-woo-products__inner-box")))
    perros_elements = driver.find_elements(By.CSS_SELECTOR, "div.jet-woo-products__inner-box")
    print(f"🐾 Procesando {len(perros_elements)} perros en esta página.")

    for i in range(len(perros_elements)):
        try:
            perros_elements = driver.find_elements(By.CSS_SELECTOR, "div.jet-woo-products__inner-box")
            driver.execute_script("arguments[0].scrollIntoView();", perros_elements[i])
            time.sleep(1)

            link = perros_elements[i].find_element(By.TAG_NAME, "a")
            ActionChains(driver).move_to_element(link).click().perform()

            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1.elementor-heading-title")))
            time.sleep(1.5)

            perfil = extraer_datos_perfil(driver)
            datos_perros.append(perfil)
            print(f"✅ {perfil['nombre']} extraído correctamente")

            driver.back()
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.jet-woo-products__inner-box")))
            time.sleep(1.5)

        except Exception as e:
            print(f"❌ Error con perro {i}: {e}")
            driver.back()
            time.sleep(1.5)

# Loop de paginación
datos_perros = []
pagina = 1

while True:
    print(f"\n📄 Scrapeando página {pagina}...")
    scrape_pagina_actual()

    try:
        pagination_buttons = driver.find_elements(By.CSS_SELECTOR, "div.jet-filters-pagination__link")
        sig_button = None
        for btn in pagination_buttons:
            if "Sig" in btn.text.strip():
                sig_button = btn
                break

        if sig_button:
            driver.execute_script("arguments[0].scrollIntoView();", sig_button)
            time.sleep(1)
            sig_button.click()
            pagina += 1
            time.sleep(2)
        else:
            print("🏁 No hay más páginas. Scraping finalizado.")
            break

    except Exception as e:
        print("⚠️ Error buscando botón 'Sig':", e)
        break

# Guardar CSV
df = pd.DataFrame(datos_perros)
df.to_csv("perfiles_perros_nuevavida.csv", index=False)
print("\n📁 Datos guardados en 'perfiles_perros_nuevavida.csv'")

✅ Cookies aceptadas.

📄 Scrapeando página 1...
✅ Scroll completado.
🐾 Procesando 16 perros en esta página.
✅ Comotú extraído correctamente
✅ Bill/Trasto extraído correctamente
✅ Ganna extraído correctamente
✅ Zirta extraído correctamente
✅ Karin extraído correctamente
✅ Lía extraído correctamente
✅ Hada extraído correctamente
✅ Eliot extraído correctamente
✅ Alma/Kyra extraído correctamente
✅ Nat extraído correctamente
✅ Polvorilla extraído correctamente
✅ Rosita extraído correctamente
✅ Pepote extraído correctamente
✅ Trufa extraído correctamente
✅ Yanko extraído correctamente
✅ Jali extraído correctamente

📄 Scrapeando página 2...
✅ Scroll completado.
🐾 Procesando 16 perros en esta página.
✅ Osa Mayor extraído correctamente
✅ Turko extraído correctamente
✅ Brandy extraído correctamente
✅ Thor extraído correctamente
✅ Pecoso extraído correctamente
✅ Petri extraído correctamente
✅ Nina extraído correctamente
✅ Cady extraído correctamente
✅ Beny extraído correctamente
✅ Carloto extraído

In [18]:
import pandas as pd
df = pd.read_csv("perfiles_perros_nuevavida.csv")
df

,nombre,sexo,edad,tamaño,raza,descripcion,compatible_con_niños,url
0,Comotú,Macho,Adulto,Pequeño (6-14 kg),NaN,"Comotú es un precioso perrito, su dueño está e...",ND,https://nuevavida-adopciones.org/ficha-13618
1,Bill/Trasto,Macho,Adulto,Pequeño (6-14 kg),NaN,Necesita casa de acogida temporal.\nTe animas?...,ND,https://nuevavida-adopciones.org/ficha-10140
2,Ganna,Hembra,Cachorro,Mediano (15-25 kg),NaN,Ganna fue recogida de una familia que no la qu...,ND,https://nuevavida-adopciones.org/ficha-13617
3,Zirta,Hembra,Cachorro,Pequeño (6-14 kg),NaN,Necesita casa de acogida temporal.\nTe animas?...,ND,https://nuevavida-adopciones.org/ficha-13616
4,Karin,Hembra,Cachorro,Pequeño (6-14 kg),NaN,Necesita casa de acogida temporal.\nTe animas?...,ND,https://nuevavida-adopciones.org/ficha-13615
...,...,...,...,...,...,...,...,...
106,Taco Mex,Macho,Adulto,Grande (+ 25 kg),NaN,Nota Informativa: Aunque se encuentra en adopc...,ND,https://nuevavida-adopciones.org/ficha-11614
107,Tiger,Macho,Adulto,Grande (+ 25 kg),NaN,Nota Informativa: Aunque se encuentra en adopc...,ND,https://nuevavida-adopciones.org/ficha-8449
108,Truman,Macho,Abuelito,Grande (+ 25 kg),Nota Informativa: Aunque se encuentra en adopc...,Ref. (SKU): 11801\nCategoría Todos los perros ...,ND,https://nuevavida-adopciones.org/ficha-11801
109,Lola Lolilla,Hembra,Abuelito,Pequeño (6-14 kg),NaN,Nota Informativa: Aunque se encuentra en adopc...,ND,https://nuevavida-adopciones.org/ficha-12579


In [19]:
df.head()

,nombre,sexo,edad,tamaño,raza,descripcion,compatible_con_niños,url
0,Comotú,Macho,Adulto,Pequeño (6-14 kg),NaN,"Comotú es un precioso perrito, su dueño está e...",ND,https://nuevavida-adopciones.org/ficha-13618
1,Bill/Trasto,Macho,Adulto,Pequeño (6-14 kg),NaN,Necesita casa de acogida temporal.\nTe animas?...,ND,https://nuevavida-adopciones.org/ficha-10140
2,Ganna,Hembra,Cachorro,Mediano (15-25 kg),NaN,Ganna fue recogida de una familia que no la qu...,ND,https://nuevavida-adopciones.org/ficha-13617
3,Zirta,Hembra,Cachorro,Pequeño (6-14 kg),NaN,Necesita casa de acogida temporal.\nTe animas?...,ND,https://nuevavida-adopciones.org/ficha-13616
4,Karin,Hembra,Cachorro,Pequeño (6-14 kg),NaN,Necesita casa de acogida temporal.\nTe animas?...,ND,https://nuevavida-adopciones.org/ficha-13615


In [20]:
df.shape

(111, 8)